In [1]:
pip install seqeval==1.0.0

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
root_dir='.'
output_dir='./output'
config = {
  "mode": "train",
  "train_file":"ner_train.txt",
  "dev_file": "ner_dev.txt",
  "word_vocab_file":"word_vocab.txt",
  "tag_vocab_file":"tag_vocab.txt",
  "trained_model_name":"epoch_{}.pt".format(5),
  "output_dir_path":output_dir,
  "word_vocab_size":2160,
  "number_of_tags": 14,
  "hidden_size": 100,
  "dropout":0.2,
  "embedding_size":100,
  "max_length": 120,
  "batch_size":64,
  "epoch":20,
}

In [3]:
import os
import torch
import torch.nn as nn
from torchcrf import CRF
from seqeval.metrics import classification_report

class RNN_CRF(nn.Module):
    def __init__(self, config):
        super(RNN_CRF, self).__init__()

        # 전체 음절 개수
        self.eumjeol_vocab_size = config["word_vocab_size"]

        # 음절 임베딩 사이즈
        self.embedding_size = config["embedding_size"]

        # GRU 히든 사이즈
        self.hidden_size = config["hidden_size"]

        # 분류할 태그의 개수
        self.number_of_tags = config["number_of_tags"]

        # 입력 데이터에 있는 각 음절 index를 대응하는 임베딩 벡터로 치환해주기 위한 임베딩 객체
        self.embedding = nn.Embedding(num_embeddings=self.eumjeol_vocab_size,
                                      embedding_dim=self.embedding_size,
                                      padding_idx=0)

        self.dropout = nn.Dropout(config["dropout"])

        # Bi-GRU layer
        self.bi_gru = nn.GRU(input_size = self.embedding_size,
                             hidden_size= self.hidden_size,
                             num_layers=1,
                             batch_first=True,
                             bidirectional=True)

        # CRF layer
        self.crf = CRF(num_tags=self.number_of_tags, batch_first=True)

        # fully_connected layer를 통하여 출력 크기를 number_of_tags에 맞춰줌
        # (batch_size, max_length, hidden_size*2) -> (batch_size, max_length, number_of_tags)
        self.hidden2num_tag = nn.Linear(in_features=self.hidden_size*2, out_features=self.number_of_tags)

    def forward(self, inputs, labels=None):
        # (batch_size, max_length) -> (batch_size, max_length, embedding_size)
        eumjeol_inputs = self.embedding(inputs)

        encoder_outputs, hidden_states = self.bi_gru(eumjeol_inputs)

        # (batch_size, curr_max_length, hidden_size*2)
        d_hidden_outputs = self.dropout(encoder_outputs)

        # (batch_size, curr_max_length, hidden_size*2) -> (batch_size, curr_max_length, number_of_tags)
        logits = self.hidden2num_tag(d_hidden_outputs)

        if(labels is not None):
            log_likelihood = self.crf(emissions=logits,
                                      tags=labels,
                                      reduction="mean")

            loss = log_likelihood * -1.0

            return loss
        else:
            output = self.crf.decode(emissions=logits)

            return output

In [4]:
from tqdm import tqdm
import numpy as np
def load_vocab(f_name):
    vocab_file = open(os.path.join(root_dir, f_name),'r',encoding='utf8')
    print("{} vocab file loading...".format(f_name))

    # default 요소가 저장된 딕셔너리 생성
    symbol2idx, idx2symbol = {"<PAD>":0, "<UNK>":1}, {0:"<PAD>", 1:"<UNK>"}

    # 시작 인덱스 번호 저장
    index = len(symbol2idx)
    for line in tqdm(vocab_file.readlines()):
        symbol = line.strip()
        symbol2idx[symbol] = index
        idx2symbol[index]= symbol
        index+=1

    return symbol2idx, idx2symbol

def convert_data2feature(data, symbol2idx, max_length=None):
    feature = np.zeros(shape=(max_length), dtype=np.int)
    words = data.split()

    for idx, word in enumerate(words[:max_length]):
        if word in symbol2idx.keys():
            feature[idx] = symbol2idx[word]
        else:
            feature[idx] = symbol2idx["<UNK>"]
    return feature

# 파라미터로 입력받은 파일로부터 tensor객체 생성
def load_data(config, f_name, word2idx, tag2idx):
    file = open(os.path.join(root_dir, f_name),'r',encoding='utf8')

    # return할 문장/라벨 리스트 생성
    indexing_inputs, indexing_tags = [], []

    print("{} file loading...".format(f_name))

    # 실제 데이터는 아래와 같은 형태를 가짐
    # 문장 \t 태그
    # 세 종 대 왕 은 <SP> 조 선 의 <SP> 4 대 <SP> 왕 이 야 \t B_PS I_PS I_PS I_PS O <SP> B_LC I_LC O <SP> O O <SP> O O O
    for line in tqdm(file.readlines()):
        try:
            id, sentence, tags = line.strip().split('\t')
        except:
            id, sentence = line.strip().split('\t')
        input_sentence = convert_data2feature(sentence, word2idx, config["max_length"])
        indexing_tag = convert_data2feature(tags, tag2idx, config["max_length"])

        indexing_inputs.append(input_sentence)
        indexing_tags.append(indexing_tag)
    indexing_inputs = torch.tensor(indexing_inputs, dtype=torch.long)
    indexing_tags = torch.tensor(indexing_tags, dtype=torch.long)

    return indexing_inputs, indexing_tags

# tensor 객체를 리스트 형으로 바꾸기 위한 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()


In [5]:
from torch.utils.data import (DataLoader, TensorDataset)
import torch.optim as optim

def train(config):
    # 모델 객체 생성
    model = RNN_CRF(config).cuda()
    # 단어 딕셔너리 생성
    word2idx, idx2word = load_vocab(config["word_vocab_file"])
    tag2idx, idx2tag = load_vocab(config["tag_vocab_file"])

    # 데이터 Load
    train_input_features, train_tags = load_data(config, config["train_file"], word2idx, tag2idx)
    test_input_features, test_tags = load_data(config, config["dev_file"], word2idx, tag2idx)

    # 불러온 데이터를 TensorDataset 객체로 변환
    train_features = TensorDataset(train_input_features, train_tags)
    train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

    test_features = TensorDataset(test_input_features, test_tags)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=config["batch_size"])

    # 모델을 학습하기위한 optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    accuracy_list = []
    for epoch in range(config["epoch"]):
        model.train()
        losses = []
        for step, batch in enumerate(train_dataloader):
            # .cuda()를 이용하여 메모리에 업로드
            batch = tuple(t.cuda() for t in batch)
            input_features, labels = batch

            # loss 계산
            loss = model.forward(input_features, labels)

            # 변화도 초기화
            optimizer.zero_grad()

            # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
            loss.backward()

            # 모델 내부 각 매개변수 가중치 갱신
            optimizer.step()

            if (step + 1) % 50 == 0:
                print("{} step processed.. current loss : {}".format(step + 1, loss.data.item()))
            losses.append(loss.data.item())



        print("Average Loss : {}".format(np.mean(losses)))

        # 모델 저장
        torch.save(model.state_dict(), os.path.join(config["output_dir_path"], "epoch_{}.pt".format(epoch + 1)))

        do_test(model, test_dataloader, idx2tag)



def test(config):
    # 모델 객체 생성
    model = RNN_CRF(config).cuda()
    # 단어 딕셔너리 생성
    word2idx, idx2word = load_vocab(config["word_vocab_file"])
    tag2idx, idx2tag = load_vocab(config["tag_vocab_file"])


    # 저장된 가중치 Load
    model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["trained_model_name"])))

    # 데이터 Load
    test_input_features, test_tags = load_data(config, config["dev_file"], word2idx, tag2idx)

    # 불러온 데이터를 TensorDataset 객체로 변환
    test_features = TensorDataset(test_input_features, test_tags)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=config["batch_size"])
    # 평가 함수 호출
    do_test(model, test_dataloader, idx2tag)

def do_test(model, test_dataloader, idx2tag):
    model.eval()
    predicts, answers = [], []
    for step, batch in enumerate(test_dataloader):
        # .cuda() 함수를 이용하요 메모리에 업로드
        batch = tuple(t.cuda() for t in batch)

        # 데이터를 각 변수에 저장
        input_features, labels = batch

        # 예측 라벨 출력
        output = model(input_features)

        # 성능 평가를 위해 예측 값과 정답 값 리스트에 저장
        for idx, answer in enumerate(tensor2list(labels)):
            answers.extend([idx2tag[e].replace("_", "-") for e in answer if idx2tag[e] != "<SP>" and idx2tag[e] != "<PAD>"])
            predicts.extend([idx2tag[e].replace("_", "-") for i, e in enumerate(output[idx]) if idx2tag[answer[i]] != "<SP>" and idx2tag[answer[i]] != "<PAD>"] )
    
    # 성능 평가
    print(classification_report(answers, predicts))


In [6]:
##########################################################
#                                                        #
#        평가 기준이 되는 지표는 Macro F1 Score          #
#           제출 포맷은 id \t predict_tag                #
#            25 \t B_PS I_PS <SP> O O O ...              #
#                                                        #
##########################################################


import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)


 34%|███▍      | 2481/7319 [00:00<00:00, 24801.21it/s]

word_vocab.txt vocab file loading...
tag_vocab.txt vocab file loading...
ner_train.txt file loading...


100%|██████████| 995/995 [00:00<00:00, 24507.64it/s]


ner_dev.txt file loading...
50 step processed.. current loss : 18.239185333251953
100 step processed.. current loss : 13.330046653747559
Average Loss : 33.474924808999766


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:45: UserWarning: <PAD> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:45: UserWarning: <SP> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

          DT       0.56      0.57      0.56       609
          LC       0.58      0.44      0.50       534
          OG       0.51      0.33      0.40       963
          PS       0.57      0.40      0.47       733
          TI       0.27      0.22      0.25        94

   micro avg       0.53      0.41      0.47      2933
   macro avg       0.50      0.39      0.44      2933
weighted avg       0.54      0.41      0.47      2933

50 step processed.. current loss : 9.317352294921875
100 step processed.. current loss : 12.180880546569824
Average Loss : 11.343710152999215
              precision    recall  f1-score   support

          DT       0.75      0.69      0.72       609
          LC       0.62      0.55      0.58       534
          OG       0.60      0.45      0.51       963
          PS       0.71      0.51      0.59       733
          TI       0.67      0.60      0.63        94

   micro avg       0.66      0.54      0.59

50 step processed.. current loss : 1.1999731063842773
100 step processed.. current loss : 1.6499300003051758
Average Loss : 1.4051212507745494
              precision    recall  f1-score   support

          DT       0.76      0.72      0.74       609
          LC       0.70      0.66      0.68       534
          OG       0.68      0.59      0.63       963
          PS       0.72      0.65      0.69       733
          TI       0.76      0.69      0.72        94

   micro avg       0.71      0.65      0.68      2933
   macro avg       0.72      0.66      0.69      2933
weighted avg       0.71      0.65      0.68      2933

50 step processed.. current loss : 0.8639364242553711
100 step processed.. current loss : 1.6236457824707031
Average Loss : 1.3075304902118186
              precision    recall  f1-score   support

          DT       0.78      0.74      0.76       609
          LC       0.74      0.63      0.68       534
          OG       0.65      0.61      0.63       963
        

In [12]:
from feature.gazetteer import GazetteFeature as GZ
gz = GZ()
gz.vocab()
ga.label2idx(

In [2]:
gz_f.vocab

{'\ufeffac밀란': {'OG'},
 '정성균': {'PS'},
 '돈내코코스': {'LC'},
 '00월초까지': {'DT'},
 '제주시': {'LC'},
 '라마다플라자청주호텔': {'OG'},
 '지구': {'LC'},
 '광주시': {'LC'},
 '이종열': {'PS'},
 '지관': {'PS'},
 '피닉스선즈': {'OG'},
 '주방천': {'LC'},
 '다음달': {'DT'},
 '고용노동부': {'OG'},
 '중국팀': {'OG'},
 '매니지먼트사': {'OG'},
 '내년0월까지': {'DT'},
 '노라존스': {'PS'},
 '전남드래곤즈': {'OG'},
 '한국': {'LC', 'OG'},
 '내년0월0일부터': {'DT'},
 '뉴로테크': {'OG'},
 '싱가포르': {'LC'},
 '모비스': {'OG'},
 '신문로': {'LC'},
 '캘빈클라인': {'OG'},
 '프랑스': {'LC'},
 '00시간': {'TI'},
 '중진협의체': {'OG'},
 '벤큐': {'OG'},
 '상계10동': {'LC'},
 '락샥': {'OG'},
 '일본해': {'LC'},
 '이장연': {'PS'},
 '0일00일부터': {'DT'},
 '박원재': {'PS'},
 '김보일': {'PS'},
 '김황식': {'PS'},
 '서산': {'LC'},
 '내곡동': {'LC'},
 '이중원': {'PS'},
 '김포공항': {'LC'},
 '샌디에이고파드리스': {'OG'},
 '박은준': {'PS'},
 '한강': {'LC'},
 'sk브로드밴드': {'OG'},
 '최순영': {'PS'},
 '(주)구성산업개발': {'OG'},
 '폴고이도스': {'PS'},
 '한국통신': {'OG'},
 '웨스트햄': {'OG'},
 '황우여': {'PS'},
 '네바다주': {'LC'},
 '정만호': {'PS'},
 '다음달00-00일': {'DT'},
 '문태호': {'PS'},
 '한진택배': {'OG'},
 '펠레': {'

In [3]:
gz_f.label2idx

{'<SP>': 0, 'DT': 1, 'LC': 2, 'OG': 3, 'PS': 4, 'TI': 5, 'O': 6}

In [4]:
gz_f.transform_word('하키')
gz_f.transform_word('중국팀')

array([0., 0., 0., 1., 0., 0., 0.])

In [5]:
gz_f.transform_line(['하키', '중국팀'])

array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.]])

In [6]:
gz_f.transform([['하키', '중국팀'], ['재무부', '사과']])

array([[[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]])

In [7]:
gz_f.transfrom_charaterized_line(['나','는','<SP>','한','국','사','람','<SP>','이','다'], 4)

array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 3.],
       [0., 0., 1., 0., 1., 0., 3.],
       [0., 0., 1., 0., 1., 0., 3.],
       [0., 0., 1., 0., 1., 0., 3.],
       [0., 0., 1., 0., 1., 0., 3.],
       [0., 0., 0., 0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [11]:
gz_f.transfrom_charaterized([['나','는','<SP>','한','국','사','람','<SP>','이','다', '<PAD>'], 
                                  ['삼','성','전','자','<SP>','이','재','용', '<SP>','회','장']], 4)

array([[[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 3.],
        [0., 0., 1., 0., 1., 0., 3.],
        [0., 0., 1., 0., 1., 0., 3.],
        [0., 0., 1., 0., 1., 0., 3.],
        [0., 0., 1., 0., 1., 0., 3.],
        [0., 0., 0., 0., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 2.],
        [0., 0., 0., 1., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 1.]]])